In [2]:
import os, re, sys
from datetime import date
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser as utp
import scripts.tools.redmine_cache as rmc

iie = importlib.reload(iie)
utp = importlib.reload(utp)
rmc = importlib.reload(rmc)

UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

In [3]:
_cache = None
_age = None
_client = None

def _get_client():
    global _client
    if _client is None:
        _client = rmc.RedmineClient()
    return _client

def _download_issues():
    _client = _get_client()
    return _client.get_filtered([('project.identifier', '==', 'pbssoftware')])

def load_issues():
    return _download_issues()


In [4]:
import re
class WebFRSIssuetrackerParser(iie.IssuetrackerParser):
    def __init__(self, types=("URS", "FRS", "SDS", "SWDS")):
        types = list(types) + ["3.0WebFRS"]
        super().__init__(types)
        
    def _get_result_for_line(self, line):
        """ identical to parent function, but checks the type after
        scanning the line to return only 3.0WebFRS items, converted
        to plain FRS items. 
        """
        if not line or line.isspace():
            return self._EMPTY_LINE, None, None, None, True

        m = self._item_match(line)
        if m is None: 
            return self._RAW_LINE, "", "", line.strip(), False
        
        dash1, typ, num, text, dash2 = m.groups()
        if typ != '3.0WebFRS':
            return self._EMPTY_LINE, None, None, None, True
        else:
            typ = 'FRS'
            num = "3." + num
        cancel = dash1 == dash2 and dash1 != ""
        return self._REQ_RESULT, typ, num, text, cancel

In [5]:
rtags = [
    "URS",
    "FRS",
    "SWDS",
    "CS",
    "BUG"
]

ttags = [
    "USR"
]

_ignore = {3194, 3287}
_sprints = {
    'Legacy',
    '3.0',
    '3.0.1',
    '3.0.2',
    '3.0.3',
    '3.0.4',
    '3.0.5',
    '3.0.6',
    '3.0.7'
}

def relevant(i):
    return i.sprint_milestone.name in _sprints and i.id not in _ignore and i.status != "Rejected"

In [25]:
iie = importlib.reload(iie)
utp = importlib.reload(utp)
UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

force = True
force = False
issues = load_issues()
issues2 = list(filter(relevant, issues.values()))
parser = iie.IssuetrackerParser(rtags)
reqs = parser.parse_all(issues2)

parser2 = UserTestsParser()
tests = parser2.parse_excel("C:\\Users\\Nathan\\Documents\\Dropbox\\FRS\\FRS Tests 190226.xlsx")

parser3 = WebFRSIssuetrackerParser(rtags)
reqs2 = parser3.parse_all(issues2)

allitems = reqs + tests + reqs2

def missing_parents(req, ref, reqs):
    if req.type == "TEST":
        return 'ignore'
    else:
        return 'fix'

rex = RequirementExtracter(rtags, ttags)
rex.config.missing_parents = missing_parents
rows = rex.extract(allitems)

In [7]:
import networkx as nx

In [22]:
all2 = []
seen = set()
for item in allitems:
    if not item.obs:
        all2.append(item)
        
for item in all2:
    if item.tag in seen:
        print(item.tag)
    seen.add(item.tag)

In [27]:
G= nx.DiGraph()

for req in allitems:
    G.add_node(req.tag)
    for ref in req.refs:
        G.add_edge(ref, req.tag)
        
nx.write_graphml(G, "text.xml")

In [19]:
nx.DiGraph.add_edge?